# Core Gene AMR Models

Este notebook é a reprodução do seguinte repositório:

https://github.com/jimdavis1/Core-Gene-AMR-Models

Aqui encontram-se os scripts necessários para construir o k-mer e modelos baseados em alinhamento para:
- Klebsiella
- Mycobacterium
- Salmonella
- Staphylococcus

## Diretórios

Automate: script que automatiza o treino para Klebsiella, Mycobacterium, Salmonella, and Staphylococcus datasets.

GenomicModelCreator: treino dos modelos e execução das estatísticas e métricas de um modelo usando dados do genoma e metadados. 

Subtree-Analysis: análise das árvores baseado no alinhamento de genes conservados.

Scripts: scripts específicos para criar alinhamentos one-hot makeAlignments.py.
makeMatrix.py pode ser usado para criar uma matriz, porém, GenomicModelCreator  faz isso por si só inicialmente, assim esse script é ignorado.

## Dados

Os dados foram obtidos de ftp://ftp.patricbrc.org/datasets/. Colocar em MeusProjetos

Na pasta, executar no cmd:

git submodule init

git submodule update

Na pasta há quatro diretórios: Klebsiella, Mycobacterium, Salmonella, and Staphylococcus. A estrutura de cada diretório pode ser descrita como:

- clades_use : clados usados para gerar modelos baseados na filogenia
- fasta.X.Y : X representa o número de genes escolhidos para o conjunto de genes, Y é a replicata de uma amostra (existem múltiplas replicatas para um conjunto de 100 genes). Estes são os mesmos conjuntos usados no paper. Haverá um arquivo fasta por genoma nomeados com a convenção genome_id.fasta.
- fasta.low10 : os modelos foram criados usando os 10 conjuntos de 100 genes e os 10 genes com a menor importância foram escolhidos. Estes são esses genes e os arquivos fasta correspondentes por genoma. Haverá um arquivo fasta por genoma nomeados com a convenção genome_id.fasta.
- *.sir.filt.plf.tab : são os metadados brutos de AMR para cada espécie.


Obs: os diretórios contendo os arquivos fasta podem não conter o mesmo conjunto de genomas, porque genomas que não contêm todos os X genes maiores que metade do tamanho médio e menores que duas vezes o tamanho médio foram removidos.

## Automate.sh

Este script serve para automatizar a construção dos modelos baseados em alinhamento a partir dos alinhamentos do repositório. Utiliza o submódulo GenomicModelCreator.

O submódulo GenomicModelCreator necessita de alguns requerimentos:

- KMC deve estar instalado e os executáveis kmc, kmc_dump e kmc_tools devem estar no caminho.

https://sun.aei.polsl.pl//REFRESH/index.php?page=projects&project=kmc&subpage=download


- Python 2.7 e pacotes devem ser instalados: numpy, sklearn, scipy, xgboost. 

- Garanta que o diretório KMC em GenomicModelCreator esteja nos caminhos. Ele só aparece com a inicialização e atualização dos submódulos

### Executando:

Com tudo pronto e trabalhando na pasta MeusProjetos, rodar no bash:

bash /pathtoautomate/automate.sh \<threads\>
    
threads é um argumento opcional onde definimos o número de threads. o default é 1.

Como já temos os dados, as linhas 51-54 estão comentadas:

echo "Downloading data..."
curl ftp://ftp.patricbrc.org/datasets/Nguyen_et_al_2020.tar.gz > $oDir/Nguyen_et_al_2020.tar.gz
echo "Decompressing data..."
tar -xzf $oDir/Nguyen_et_al_2020.tar.gz -C $oDir

Then it will begin building multiple types of models for Staphylococcus (Klebsiella, Mycobacterium, and Salmonella can also be added, however). These models will be stored in the models directory that will be created inside the specified output directory. The following types of models will be built:

Alignment-based models
Clade-weighted models (no weight, weight by clade size, weight by S/R distribution in clade, weight by both)
Gene sets (10 replicates of 100 genes, 1 replicate of 25, 50, 250, and 500 genes)
Shuffled labels (10 replicates of 100 genes)
Note that the output directory does not need to reside within the repo and the bash script can be called from outside the repo as well.

Script Output
Inside the output directory of the script there will be a some directories:

models Directory: Contains 1 directory per species which hold the respective models for each species.
Nguyen_et_al_2020 Directory: The raw data used to run the models. This raw data matches that which was used in the paper.
Nguyen_et_al_2020.tar.gz File: Tarball which was downloaded and extracted to the Nguyen_et_al_2020 directory.
rand.tab File: Generated on the fly, random shuffling of the last run shuffled labels model.
temp Directory: A directory used to store temporary data while building models. This can be cleared.
The models directory will contain 4 directories (one for each species) which each contain multiple models which have the following within their directory structure:

all : directory containing the 5 of 10 folds run. Each fold contains a .pred, .train_hist.txt, .true, .tree, and pkl file representing the predicted labels for the test set, training history (merror for train, validation, and test), true values for the testing set, text representation of the tree ensemble, and the model itself in pkl format. Additionally, statistic files of metrics also are included confusion_matrix.tab, f1.tab, raw_acc.tab, and VMEME.tab. model.params is included for the XGBoost parameters used in the model.
model.attrOrder : attribute order for the features of the model.
model.genomes.list : list of genomes used to train the model.
model.labels.map : label mapping
model.params : parameters sent to model script
temp.txt : temprary file (used for debugging)
weights.list : list of weights (used for debugging)